In [1]:
import pandas as pd
from IPython.display import Markdown
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold

In [2]:
df = (
    pd
    .read_csv('../data/external/data.csv')
)
df = df.assign(
    site = df.site.str.split("/").apply(lambda x: f"{x[0]}//{x[2]}")
)
display(Markdown("### Conjunto de dados"))
display(df.head())

df_dict = pd.read_csv('../data/external/dicionario.csv')
display(Markdown("### Dicionário de dados"))
display(df_dict.head())

### Conjunto de dados

,text,datetime,category,label,site
0,Kátia Abreu diz que vai colocar sua expulsão e...,2017-11-30,politica,fake,https://ceticismopolitico.com
1,"Dr. Ray peita Bolsonaro, chama-o de conservad...",2017-11-24,politica,fake,https://ceticismopolitico.com
2,Reinaldo Azevedo desmascarado pela Polícia Fed...,2017-05-23,politica,fake,https://afolhabrasil.com.br
3,Relatório assustador do BNDES mostra dinheiro ...,24/07/2017,politica,fake,https://www.diariodobrasil.org
4,"Radialista americano fala sobre o PT: ""Eles ve...",25/07/2017,politica,fake,https://www.diariodobrasil.org


### Dicionário de dados

,variavel,significado,tipo,valores
0,text,"Coluna contendo o texto das matérias, sendo ca...",text,string
1,datetime,Coluna contendo a data de publicação das matérias,time,datetime
2,category,Coluna contetndo a categoria na qual cada maté...,nominal,"['politica', 'tv_celebridades', 'sociedade_cot..."
3,label,Coluna contendo os rótulos se a notícia é fals...,nominal,"['fake', 'true']"
4,site,Coluna contendo o link do site em que a notíci...,nominal,string


In [3]:
taget_column = "label"
nominal_columns = (
    df_dict
    .query(
        "tipo == 'nominal' and variavel != @taget_column"
    )
    .variavel
    .to_list()
)
text_columns = (
    df_dict
    .query(
        "tipo == 'text' and variavel != @taget_column"
    )
    .variavel
    .to_list()
)
time_columns = (
    df_dict
    .query(
        "tipo == 'time' and variavel != @taget_column"
    )
    .variavel
    .to_list()
)

In [4]:
nominal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ("missing", SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ("encoder", OneHotEncoder(sparse=False)), # Codificação de variáveis
    ("selector", VarianceThreshold(threshold=.1)), # Seleção de variáveis
    ("normalizer", StandardScaler()), # Normalização
])
